操作系统学习——进程管理

#### 操作系统的功能
操作系统的功能主要有九种：
- System initiation 在系统启动并且初始化的时候将对应的操作系统内容装载进内存
- ***管理内存、进程***
- 中断管理（ interupt）
- 定时器（ 一种特殊的中断，CPU会将时间分割成十毫秒的时钟切片，当每个切片处理完之后也会触发中断）
- 文件接口
- 驱动管理
- ***进程通信***
- 网络通信

##### ***进程及内存管理：*** 

操作系统的进程分为两部分，***外部的用户代码和内部的内核***

用户进程通过系统调用将命令传递给内核态的进程，再通过内核与对应的硬件进行通信（比如磁盘，鼠标，屏幕
***这里记得插张图片

#### 内核架构
操作系统的内核架构分为两种:
- 一种是最为常见的单一内核，特点是在操作系统初始化的时候将所有功能装载到内存空间
- 一种是微内核，特点是只将必要的内容装载在内存空间，剩余的内容放到磁盘空间，多见于一些功能简单，内存较小的操作系统
- linux 所使用的内核架构类似微内核和单一内核的折中产物，部分功能不会在操作系统初始化时被装载而是以模组形式保存在磁盘空间，需要时再装载进内核之中

#### 内存空间布局
- 在内存为4GB的三十二位操作系统中，最低端的地址被分配给了系统的room
- 高位地址，OXC0000……0 到FFFFF……F, 也就是最末尾的1GB被分配给了操作系统的内核
- 中间剩下的地址，被分配给了用户程序。
- Task_size 标志着内核地址和用户地址的分界线（使用task_size 作为分界标记而不是直接规定代码段，为了方便后续修改内存分配不至于出错
在用户空间之中, 从高位到低位地址分别是：
- 栈（stack
- 文件映射
- 堆（heap
- DATA VMA 
- Code VMA\
注意 VMA是虚拟内存描述符，它只存在与逻辑之中，在实际程序运行时VMA会被替换为实际的物理地址
![table](loading_the_data.jpg)

### ****进程控制块/task_struct****
处于内核的一个结构变量，存储了对应的用户进程的堆栈位置和内存影像
- 所有线程共享几乎所有内容
- 线程在task_struct里不同，但是指向同一个mm_struct 
- 两个线程同时共享一个控制块，他们会使用同一个堆，但是其中一个的栈是从另一个的堆上划分出来的
- 进程控制块连接到本进程的空间描述符再链接到vma描述符
![](task_struct.jpg)

从mm_struct 的映射中就可以看出`p0`, `p1`,`p2` 分别是不同的进程，`p2-1`和`p2-2`是同一进程下的两个线程。
我们可以看出线程虽然有单独的控制块，但是他们所映射到的内存影像是同一个
以及虽然两个线程共用存储，但是他们的堆栈是相对独立的
![](multiple_task_struct.jpg)

pthread 线程， process 进程
Pthread_create函数，Pthread_join函数，Pthread_self函数，Pthread_detach函数，Pthread_exit函数
detach 可汇合，或者可脱离的线程

### 进程通信 ###
在linux系统之中各个并发进程之间是互相隔离的，他们拥有自己的地址空间并且无法直接访问对方的地址空间。但是实际程序运行之中需要进程之间互相通信—— 例如共享变量，等待或者协调资源的使用。所以造作系统需要提供灵活高效的同步和通信手段
线程通讯相对简单，它包括：
- Volatile全局变量（条件变量
- Mutext
- 互斥锁
- semaphore


### **方法一 共享内存** ###
实现方式：将两个或以上的进程部分的虚存空间映射到相同的物理空间。
- 注意，即使这个技术不需要涉及内核空间的分配，申请共享内存仍然在内核完成

### **方法二 将内核作为数据的中转途径** ###
实现方式： 在内核中划出来一块存储空间
## 管道
适合生产者producer和消费者consumer之间的单向通信，也就是一方读一方写的模式， 分为两种
- 无名管道：利用文件描述符的继承进行访问，只能在父子进程之间进行访问，不能实现任意进程间通信
- 有名管道（FIFO）：可以实现多个不相干进程之间的通信

## Semaphore
一种分配通信通道资源的体系
当可用资源数等于零的时候，semaphore = 0 正在等待资源的进程进入睡眠状态
当资源使用完毕，并且回收之后，semaphore将按照优先级顺序唤醒一个正在等待的进程（最高优先级进程)并将资源分配给它

## 消息队列
和取快递有那么点像。
每个收发信息的进程都将创建一个结构体，分别在q_message  q_receive和 q_sender三个链表之中。当消息队列收到一个消息之后， 它将根据消息类型唤醒对应的进程。当信息传递完成之后释放对应结构体

## socket 套接字
套接字原本是用于网络通信之中收发信息，随着网络编程的发展越来越多的应用于进程间数据传输。本质上和TCP那个套接字没什么区别，只是收发对象从主机与服务端变成了两个不同的进程

## sig 宏传输
由一个进程向另一个进程发送一个三十二位的短标记
不同的数位01会有不同的解读。 
一般用于强迫进程异步处理。

### 读写保护 ###
在进行进程通信或者文件管理的时候，有些存储空间将会同时涉及到写入和读取。这些空间被称为critical region （临界区）有很多函数用来保证临界区不会出些多个进程同时写或读写的情况。

## 互斥锁(Mutilexclusion)
保证任何时刻只能有一个进程、线程执行这段代码 
Mutext(互斥锁相关函数) 封装好的互斥锁函数，需要使用的时候直接调用就可以。
多个线程被阻塞在同一个互斥锁上，那么恢复的时候不同的函数会根据将最高优先级的进程或线程唤醒。

## 条件变量 
与互斥锁类似，唯一的不同点在于条件变量提供了一个函数用于在满足条件情况下唤醒所有的阻塞进程。

## 读写锁
没有线程持有用于写的读写锁时，任意多的线程可以持有读锁
当没有线程持有读锁，才可以分配写锁（一个）


### 内核管理 ###
并发和竞态
多个应用可能会调用接口，运行中被中断打断。并发相关缺陷，最难以找到的也是最难以处理的
早期内核并发来源比较少不支持symmetric multiprocessing 对称多处理器，导致唯一并发来源是中断服务。但是现在内核伸缩性得到大幅提升， 提高内核编程的复杂度。

## 原子操作，最低最小的不可被干扰的操作
## 互斥锁
## 自旋锁spanlock
一个自旋锁是一个互斥，只有锁定和解锁。
某个整数值的单个位来表示 
整段函数不断自循环并检查这个位是否为1  如果可用则使用并改为锁定
多个自选进程也只能同时唤醒一个
和信号量不同，自旋锁可以在睡眠中使用
注意 **（中断处理程序不允许休眠所以它只能用自旋锁）
所有自旋锁均不可中断 使用自旋锁之后CPU不能切换（所以只能等）**
假定一个中断获得了自旋锁但是因为某些原因进入了等待和睡眠，剩下进程也不能打断自旋锁或者获得它导致处理器卡死。
拥有自旋锁的代码必须不能休眠，甚至中断也不能让进程放弃处理器。自旋锁会禁止任何处理器上的抢占。（关中断）
所有函数都有可能休眠。所以设置自旋锁的时候必须将所有内核考虑在内，与此同时，因为自旋锁的霸道，一般不会在里面设置过于复杂的操作以保证最短的拥有时间。拥有锁的时间越长，其它进程自旋等待的时间会越长，就越可能影响处理器调度。
## 使用锁的陷阱
	并行进程必须同时有对应的锁
锁定模式必须一开始定好
避免某个需要锁的进程需要另一个上锁进程的
任何情况下系统都不允许锁拥有者获取第二个锁（信号量，各种锁）
	
### 某些情况下原子访问不需要完整的锁（免锁算法）
某些指令天然就是原子的（他们可以在一个时钟周期内完成所以不可分割
	**从根本上避免锁**
		如果有大量读入但只有一个写
			Circular buffer，循环缓存 两个指针，一个写一个读，永远都不会同时在同一个单元上读写 多见于驱动，尤其是网络驱动
	**原子整数**
		在所有内核支持架构上，保存一个int它的加减本身是原子操作。（所有smp都能做到，因为编译器会将它编译成单个机器指令）
	**位操作**
		单独对某个二进制位进行操作,也是原子操作
	**Seq lock（简单版读写锁**
		保证读的自由访问，（但是需要读入者检查是否冲突）如果读写冲突则重新读
asymmetric processing 斜处理器（一大一小 ，一主一副）例如一个主处理器和一个数字信号处理器
	**读取复制更新 ReadCopyUpdate**
···		高级互斥机制，很少在驱动中使用。针对长期读取但是很少写入的数据进行了优化。被保护的资源只能通过指针访问，并且这个引用必须只有原子代码拥有（保护的资源变成了指针指向的对象。）由于指针是一字节更新指针是一个原子代码，所以不需要额外加锁。***读操作优先大于写***
e.g 网络路由表，
		在修改时，写入程序先是复制一个副本，写入之后再替换相关指针。
		
